<a href="https://colab.research.google.com/github/stick35em10/Teste-A-B/blob/main/notebooks/teste_AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalations
#!pip install -r requirementes.txt
!pip install pandas

In [ ]:
# instalations
!pip install statsmodels

# **0.0. Import packages**

In [3]:
# imports
import pandas as pd
from google.colab import drive


In [4]:
import statsmodels.stats.power as sms
import math

# **1.0. Load data**

In [24]:
#!pwd
#!ls /content/drive/MyDrive/'Colab Notebooks'/'Teste AB'/dataset/ab_data.crdownload #/sample_data/
#df_raw.shape: (18583, 5)
#path_ab_data = '/content/drive/MyDrive/Colab Notebooks/Teste AB/dataset/ab_data.crdownload' #/sample_data/'
#df_raw.shape: (294478, 5)
path_ab_data = '/content/drive/MyDrive/Colab Notebooks/Teste AB/dataset/ab_data_.csv'


In [25]:
#load data
df_raw=pd.read_csv(path_ab_data) #'/content/drive/MyDrive/Colab Notebooks/Teste AB/dataset/ab_data.crdownload')
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [26]:
df_raw.shape

(294478, 5)

# **2.0. Design de Experimentos**

### **2.1. Formulação de Hipóteses**

### **2.2. Parâmetros do Experimento**

In [11]:
# nivel de confianca
confidence_level    = 0.95

# nivel de significancia
significance_level  = 0.05

# conversao da pagina actual e da nova pagina
p1 = 0.13 # medido
p2 = 0.15 # esperado pelo time de negocio

# tanho do efeito
# AttributeError: module 'statsmodels.stats.power' has no attribute 'proportion_effectsize'
#ffect_size = sms.proportion_effectsize(p1, p2) #AttributeError: module 'statsmodels.stats.power' has no attribute 'proportion_effectsize'

# The function 'proportion_effectsize' is deprecated.
# Instead, use 'compute_effect_size' from 'statsmodels.stats.proportion'
from statsmodels.stats.proportion import proportion_effectsize
effect_size = proportion_effectsize(p1, p2)

# tanho do efeito
# effect_size = sms.proportion_effectsize(p1, p2) #The function 'proportion_effectsize' is no longer in 'statsmodels.stats.power'
# Instead, use 'compute_effect_size' from 'statsmodels.stats.proportion'
#from statsmodels.stats.proportion import compute_effect_size
#effect_size = compute_effect_size(p1, p2)

import numpy as np

# effect_size = sms.proportion_effectsize(p1, p2) #This function might be deprecated
# Instead, try using the following:
from statsmodels.stats.power import TTestIndPower, NormalIndPower
#effect_size = (p2 - p1) / np.sqrt(p1 * (1 - p1)) # Calculate effect size manually

# Poder estatística
power = 0.80 # erro de oportunidade

# sample size
#AttributeError: 'NormalIndPower' object has no attribute 'solver_power'
#sample_n = sms.NormalIndPower().solver_power(
sample_n = NormalIndPower().solve_power(
    effect_size=effect_size,
    power=power,
    alpha=significance_level
) #4720
#nobs1=None,
#ratio=1 # Assuming equal sample sizes for both groups

##4721
#sample_n = TTestIndPower().solve_power(
#    effect_size=effect_size,
#    power=power,
#    alpha=significance_level,
#    nobs1=None,
#    ratio=1
#)#4721

sample_n = math.ceil(sample_n)
sample_n # amostras para cada um dos grupos

4720

# 3.0 Colecta dos dados

## 3.1.Análise descritiva dos dados

In [12]:
# 2.0 Análise descritiva dos dados
print(f'Number of rows: {df_raw.shape[0]}')
print(f'Number of columns: {df_raw.shape[1]}')
#print(f'Columns: {df_raw.columns.to_list()}

Number of rows: 294478
Number of columns: 5


## 3.2.Verificação dos dados faltantes, check NA

In [19]:
df_raw.isna().sum()

,0
user_id,0
timestamp,0
group,0
landing_page,0
converted,0


## 3.2.Conferir as "flags"

In [42]:
#len(
df_raw[['user_id','group','landing_page']].groupby(['group','landing_page']).count().reset_index() #.query('group == 2 and landing_page == 2')#)

,group,landing_page,user_id
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


In [81]:
#df_raw[['user_id','group','landing_page']].groupby(['user_id']).count().reset_index().query("group == 'control' and (landing_page == 'new_page' or landing_page == 'old_page')")
linhas_com_duplicação=len(df_raw[['user_id','group','landing_page']].groupby(['user_id']).count().reset_index().query('landing_page >1'))
percentagem_duplicado=linhas_com_duplicação/df_raw.shape[0]
print(round(percentagem_duplicado*100,2), '% de linhas duplicadas')
print('com são poucos dados duplicados e ainda menor que 5% podemos deletar, ')
df_user_delete = df_raw[['user_id','group','landing_page']].groupby(['user_id']).count().reset_index().query('landing_page >1')['user_id']

#data without duplicated
df1 = df_raw[~df_raw['user_id'].isin(df_user_delete)] # .groupby(['user_id']).count().reset_index().query('landing_page >1')
df1.head()
print(df1.head(), df1.shape)

#df_raw[['user_id','group','landing_page']].reset_index().query('user_id in [630052,630126,630137]') #43,111,122]

1.32 % de linhas duplicadas
com são poucos dados duplicados e ainda menor que 5% podemos deletar, 
   user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1 (286690, 5)


In [51]:
# delectando os duplicados
df_raw_sem_duplicates = df_raw.drop_duplicates(inplace=True)
print(df_raw_sem_duplicates)
df_raw[['user_id','group','landing_page']].groupby(['user_id']).count().reset_index() #.query('group == 2 and landing_page == 2')#)


        user_id                   timestamp      group landing_page  converted
0        851104  2017-01-21 22:11:48.556739    control     old_page          0
1        804228  2017-01-12 08:01:45.159739    control     old_page          0
2        661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3        853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4        864975  2017-01-21 01:52:26.210827    control     old_page          1
...         ...                         ...        ...          ...        ...
294473   751197  2017-01-03 22:28:38.630509    control     old_page          0
294474   945152  2017-01-12 00:51:57.078372    control     old_page          0
294475   734608  2017-01-22 11:45:03.439544    control     old_page          0
294476   697314  2017-01-15 01:20:28.957438    control     old_page          0
294477   715931  2017-01-16 12:40:24.467417  treatment     new_page          0

[294478 rows x 5 columns]


In [13]:
# removendo doplicado
len(df_raw[['user_id','group']].groupby(['user_id']).count().reset_index().query('group > 1')) #['user_id']
#df_raw['user_id'

3894

In [14]:
### 2.1 Conferir as "flags " do teste A
#df_user_delete = df_raw[[ 'user_id','group']].groupby(['user_id']).count().reset_index().query('group > 1')['user_id']
#group/landig old new
df_raw[['user_id','group','landing_page']].groupby(['group','landing_page']).count().reset_index()#.query('group > 1')[['user_id','group','landing_page'

,group,landing_page,user_id
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


In [15]:
### 2.1 Conferir as flags do teste A


df_user_delete = df_raw[[ 'user_id','group']].groupby(['user_id']).count().reset_index().query('group > 1')['user_id']

# data without duplicated
df1 = df_raw[~df_raw['user_id'].isin(df_user_delete)]
df1.head()
df1.shape

(286690, 5)

In [16]:
### 2.2.Amostragem aleatoria dos grupos Controle e Tratamento
# Control Group
df_control_sample = df1[df1['group'] == 'control'].sample(n=sample_n, random_state=42)
print('Size of Control Group: {}'.format(df_control_sample.shape[0]))

# Treatment Group
df_treatment_sample = df1[df1['group'] == 'treatment'].sample(n=sample_n, random_state=42)
print('Size of Treatment Group: {}'.format(df_treatment_sample.shape[0]))

# Total sample Size
df_ad = pd.concat([df_control_sample, df_treatment_sample], axis=0)
print(df_ad.shape)

df_ad_= pd.concat([df_control_sample, df_treatment_sample]).reset_index(drop=True) #res.reindex(df_raw.index)
df_ad_.head()
print(df_ad_.shape)
#
# Treatement Group
# Total Sample sizze

Size of Control Group: 4720
Size of Treatment Group: 4720
(9440, 5)
(9440, 5)


In [17]:
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [18]:
### 2.3 Calculo da métrica de intersse entre os Grupos (conversão de cada página)
# ============================= Control Group =============================
#sales = df_control_sample.loc[df_control_sample['landing_page'] == 'old_page', 'converted'].sum()
sales = df_control_sample.loc[df_control_sample['converted'] == 1, 'converted'].sum()
visit = len(df_control_sample)

conversion_rate_control = sales / visit
print('Conversion Rate of Control Group: {}'.format(conversion_rate_control))

# ============================= Treatment Group =======================
sales = df_treatment_sample.loc[df_treatment_sample['converted'] == 1, 'converted'].sum()
visit = len(df_treatment_sample)

conversion_rate_treatment = sales / visit
print('Conversion Rate of Treatment Group: {}'.format(conversion_rate_treatment))

# conver
visit


Conversion Rate of Control Group: 0.11546610169491525
Conversion Rate of Treatment Group: 0.12902542372881357


4720